In [8]:
import os
import numpy as np
import datetime
import matplotlib.pyplot as plt
s_file="Vizag_000001_S03-2012.SDT"
#s_file = os.path.join(s_path, s_file_name)
s_out = s_file.replace('.SDT', '.his')
s_out4 = s_file.replace('.SDT', '_225.csv')
s_out5 = s_file.replace('.SDT', '_SPT.txt')
Smax=None
Lat=None
Lon=None
if os.path.exists(s_file):
    nb = 0
    nspt = 0
    with open(s_file, 'rb') as fid, open(s_out, 'w') as fod, open(s_out4, 'w') as fod4, open(s_out5, 'w') as fid_spt:
        while True:
            hdr = np.fromfile(fid, dtype=np.uint8, count=5)
            nb += 5
            tms = np.fromfile(fid, dtype=np.uint8, count=6)
            if tms.size == 0:
                break
            year = tms[0] * 256 + tms[1]
            month = tms[2]
            day = tms[3]
            hour = tms[4]
            minute = tms[5]
            dt=datetime.datetime(year, month, day, hour, minute)
            bspt = np.fromfile(fid, dtype=np.uint8, count=512)
            bsys = np.fromfile(fid, dtype=np.uint8, count=32)
            chks = np.fromfile(fid, dtype=np.uint8, count=1)
            b=np.concatenate((tms, bspt, bsys, chks))
            cs = b[0]
            for j in range(1, len(b)):
                cs ^= b[j]
            if cs == 0:
                GPS = (bsys[1] // 16) % 8
                Hm0 = ((bsys[2] * 256 + bsys[3]) % 4096) / 100
                Tz = 400 / ((bsys[4] * 256 + bsys[5]) % 256)
                Smax = np.exp(-0.005 * ((bsys[6] * 256 + bsys[7]) % 4096)) * 5000
                Tref = ((bsys[8] * 256 + bsys[9]) % 1024) / 20 - 5
                Tsea = ((bsys[10] * 256 + bsys[11]) % 1024) / 20 - 5
                Bat = bsys[12] % 8
                BLE = ((bsys[12] * 256 + bsys[13]) // 16) % 256
                Av = (bsys[14] * 256 + bsys[15]) % 4096
                Av = (2048 - Av if Av > 2048 else Av) / 800
                Ax = (bsys[16] * 256 + bsys[17]) % 4096
                Ax = (2048 - Ax if Ax > 2048 else Ax) / 800
                Ay = (bsys[18] * 256 + bsys[19]) % 4096
                Ay = (2048 - Ay if Ay > 2048 else Ay) / 800
                Lat = (((bsys[20] % 16) * 256 + bsys[21]) * 16 + (bsys[22] % 16)) * 256 + bsys[23]
                Lat = (Lat % (2**24)) / (2**23) * 90
                Lat = 90 - Lat if Lat > 90 else Lat
                Lon = (((bsys[24] % 16) * 256 + bsys[25]) * 16 + (bsys[26] % 16)) * 256 + bsys[27]
                Lon = (Lon % (2**24)) / (2**23) * 180
                Lon = 180 - Lon if Lon > 180 else Lon
                ori = ((bsys[28] * 256 + bsys[29]) % 4096) * 360 / 256
                incl = (bsys[31] + (bsys[30] % 16) / 16) * 360 / 256 / 2 - 90
                sys = [Hm0, Tz, Smax, Tref, Tsea, Bat, BLE, Av, Ax, Ay, GPS, Lat, Lon, ori, incl]
                spt = []
                for j in range(0, len(bspt), 8):
                    jf = bspt[j] % 64
                    frq = jf * 0.005 + 0.025 if jf < 16 else jf * 0.01 - 0.05
                    sprlsb = bspt[j] // 64
                    dir_angle = bspt[j + 1] * 360 / 256
                    psd = np.exp(-0.005 * ((bspt[j + 2] * 256 + bspt[j + 3]) % 4096))
                    n2lsb = (bspt[j + 2] // 16) % 4
                    m2lsb = bspt[j + 2] // 64
                    spr = (bspt[j + 4] + sprlsb / 4) * 360 / 256 / np.pi
                    m2 = (bspt[j + 5] + m2lsb / 4) / 128 - 1
                    n2 = (bspt[j + 6] + n2lsb / 4) / 128 - 1
                    K = bspt[j + 7] * 0.01
                    sgmc = spr * np.pi / 180
                    m1 = 1 - sgmc**2 / 2
                    sgmca = np.sqrt((1 - m2) / 2)
                    skw = -n2 / sgmca**3
                    kurt = (6 - 8 * m1 + 2 * m2) / sgmc**4
                    spt.append([frq, Smax * psd, dir_angle, spr,skw,kurt, m2, n2, K, Lat, Lon])
                nspt += 1
                mom = np.zeros(8)
                mom2 = np.zeros(14)
                frq = first_elements = [inner_list[0] for inner_list in spt]
                psd = first_elements = [inner_list[1] for inner_list in spt]
                o = 3
                oo = 5
                for j in range(1, len(frq)):
                    for n in range(-2, 5):
                        yj1 = psd[j - 1] * frq[j - 1]**n
                        yj = psd[j] * frq[j]**n
                        mom[o + n] += 0.5 * (yj + yj1) * (frq[j] - frq[j - 1])
                    for n in range(-4, 9):
                        yj1 = psd[j - 1]**2 * frq[j - 1]**n
                        yj = psd[j]**2 * frq[j]**n
                        mom2[oo + n] += 0.5 * (yj + yj1) * (frq[j] - frq[j - 1])

                Hm0 = 4 * np.sqrt(mom[o])
                TI = np.sqrt(mom[o - 2] / mom[o])
                TE = mom[o - 1] / mom[o]
                T1 = mom[o] / mom[o + 1]
                Tz = np.sqrt(mom[o] / mom[o + 2])
                T3 = np.sqrt(mom[o + 1] / mom[o + 3])
                T4 = (mom[o + 1] / mom[o + 4]) ** 0.25
                Gamma = mom[o + 2] * mom[o] / mom[o + 1] ** 2
                sigH = np.sqrt(mom2[oo] - mom[o] ** 2)
                skew = (mom2[oo + 1] - 3 * mom[o] ** 2) / sigH ** 3
                kurt = (mom2[oo + 2] - 4 * mom2[oo + 1] * mom[o] + 6 * mom[o] ** 2 - 3 * mom[o] ** 4) / sigH ** 4
                
                prms, prms4 =[Hm0, TI, TE, T1, Tz, T3, T4, Gamma, sigH, skew, kurt], [Hm0, TI, TE, T1]
                dts = dt.strftime("%Y-%m-%dT%H:%M:%S")

                # Format and write `prms` values individually
                fod.write(f"{dts}, {', '.join(f'{p:.2f}' for p in prms)}, "
                          f"{sys[4]:.2f}, {sys[5]:.2f}, {sys[6]}\n")
                # Format and write `prms4` values individually
                fod4.write(f"{dts}\t{'\t'.join(f'{p:.2f}' for p in prms4)}\n")
    
                #print(spt[0])   
                # Format and write `spt` values individually
                fid_spt.write(f"Time Stamp= {dts}\n")
                for row in spt:
                    fid_spt.write("\t".join(f"{val:.3f}" for val in row) + "\n")
                #fid_spt.write((f"Time Stamp= {dts}\n" + f"{x:.3f}" for x in spt) + '\n')
            nb += 551


[  7 220   3   1   0  10 192  82 215 123 167 110 120 255 193 124 134 153
 167 136 156 255  66 177  68 222 158 177 125 255   3  28  35 226 157 143
 203 255   4 150 195 204 136 158 184 255 197  97  18 226 120 192 143 255
 198 118 211   1 135 135 106 255 199 115  18 203 113 183 132 255 200  89
   2  77 101 197 147 248 137 113 193  85  82 233 124 203 202 108  80   9
  41 251 129 191 203 110 112  24  33 247 126 165 140 108 208  10  48 242
 136 140 205 115 240   0  40 245 127 183  78 105  80 229  48 243 127 162
 207 108  81  38  61 228 140 156  16 112  65 149  77 225 140 140  81 109
  98  34  71 231 138 150  82 107   2 104  84 212 152 144 211  99  50 183
 111 165 164 117 148 113 210 182  93 207 147 110  85 120 210 102  94 231
 134 101  22 118 114  91  73 238 135 126  23 116  97 255  57 235 134 110
 216 117  65 143  46 242 131 111  89 114  17 166  53 240 137  99  26 118
  33 184  52 240 131  96  91 121  17 209  52 239 136 100 156 119  81 174
  43 243 132 108 157 116 210  33  57 235 129 107  9